In [24]:
import os
import pandas as pd
import json
import re
from deep_translator import GoogleTranslator
import swifter

In [25]:
pd.options.display.max_colwidth = 500

In [46]:
dfs = []
for r, d, f in os.walk(os.getcwd()):
    for file in f:
        if 'withheldtweets.json' in file:
            dfs.append(pd.read_json("./censored_tweets/%s" % file, lines=True))

df_cen = pd.concat(dfs)
df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [47]:
df_cen.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status', 'quoted_status_permalink', 'is_quote_status',
       'extended_tweet', 'quote_count', 'reply_count', 'retweet_count',
       'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
       'lang', 'timestamp_ms', 'linked', 'display_text_range',
       'withheld_in_countries', 'extended_entities', 'possibly_sensitive',
       'retweeted_status', 'withheld_copyright'],
      dtype='object')

### create dataset with traduction

In [48]:
df_cen.set_index('id', inplace=True)

In [49]:
df_cen.shape

(41727, 38)

In [50]:
df_without_duplicate = df_cen.drop_duplicates("text")

In [51]:
df_without_duplicate.shape

(23081, 38)

In [52]:
def remove_user_mentions(tweets):
    return re.sub(r"@\w+", " ", tweets)

def remove_numbers(tweets):
	return re.sub(r"\d+([.,]\d+)?", " ", tweets)

def remove_URLs(tweets):   
    return re.sub(r'http\S+', '', tweets)

def remove_RT(tweets):
    return re.sub("RT  :", " ", tweets)

def remove_n(tweets):
    return tweets.replace("\n", "")

In [55]:
df_without_duplicate["text"].apply(lambda x: remove_user_mentions(x))

id
1365252415444946945                                                                              #Balakot \nPak Army is our pride ❤️👍 https://t.co/JirGPSMMGO
1365251727574900738          RT  : لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…
1365249991137251328           RT  : سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ  لے کر انہوں  نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…
1365242894362279938            RT  : في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى #ذكرى_مجزرة_الحرم_الإ…
1365242898569134080    RT  : Mujhe aj bhi batting ki bari ni di😔\n Yahin so jata hun! Subha jb sab ayenge pehle pehle me kheilunga🤔🤔😁 https://t.co/0PjHxburfB
                                                                                        ...                                                                  
1361903355984023552     RT  : مجھ سے ملو تو زرا ف

In [53]:
clean_tweets = df_without_duplicate["text"].apply(lambda x: remove_user_mentions(x))

In [35]:
clean_tweets = clean_tweets.apply(lambda x: remove_n(x))
clean_tweets = clean_tweets.apply(lambda x: remove_URLs(x))
clean_tweets = clean_tweets.apply(lambda x: remove_numbers(x))
clean_tweets = clean_tweets.apply(lambda x: remove_RT(x))

In [36]:
df_clean = df_without_duplicate.copy()

In [37]:
df_clean['text'] = clean_tweets

In [39]:
df_clean.shape

(23081, 38)

In [40]:
df_clean[clean_tweets.apply(lambda x: x.isspace())].shape

(530, 38)

In [41]:
df_clean

created_at               id_str  \
id                                                                   
1365252415444946945 2021-02-26 10:48:35+00:00  1365252415444946944   
1365251727574900738 2021-02-26 10:45:51+00:00  1365251727574900736   
1365249991137251328 2021-02-26 10:38:57+00:00  1365249991137251328   
1365242894362279938 2021-02-26 10:10:45+00:00  1365242894362279936   
1365242898569134080 2021-02-26 10:10:46+00:00  1365242898569134080   
...                                       ...                  ...   
1361903355984023552 2021-02-17 05:00:37+00:00  1361903355984023552   
1361917897631805442 2021-02-17 05:58:24+00:00  1361917897631805440   
1361912499562553344 2021-02-17 05:36:57+00:00  1361912499562553344   
1361916228311408640 2021-02-17 05:51:46+00:00  1361916228311408640   
1361908103915053060 2021-02-17 05:19:29+00:00  1361908103915053056   

                                                                                                                                             text  \
id                                                                                                                                                  
1365252415444946945                                                                                           #Balakot Pak Army is our pride ❤️👍    
1365251727574900738         لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ ہمارے فیس بک پیج پر   لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔اس کے…   
1365249991137251328     سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ  لے کر انہوں  نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…   
1365242894362279938         في مثل فجر هذا اليوم قبل   عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. ستبقى #ذكرى_مجزرة_الحرم_الإ…   
1365242898569134080                       Mujhe aj bhi batting ki bari ni di😔 Yahin so jata hun! Subha jb sab ayenge pehle pehle me kheilunga🤔🤔😁    
...                                                                                                                                           ...   
1361903355984023552                           مجھ سے ملو تو زرا فاصلے سے ملا کرو۔۔۔میں دل میں اتر جانے کا کمال رکتهی ہوں۔۔۔😌💕#مروہ_خان #محفل_سخن    
1361917897631805442                                                "Her zaman demeye çalışıyorum da bugünlerde daha çok söylemek içimden geldi."    
1361912499562553344                                                               Bu arada,    bir haftaya kalmaz Yunanistan ile kriz çıkartır...   
1361916228311408640    السلام علیکمبسم الله الرحمن الرحيمسيده عائشہ رضی اللہ عنہا سے روایت ہے کہ نبی صلی اللہ علیہ وسلم نے فرمایا: شریف لوگوں کی…   
1361908103915053060                                                                 मैं RSS के देशहित के हर कार्य से जुड़ा हूं - मिथुन चक्रवर्ती    

                                                                                                   source  \
id                                                                                                          
1365252415444946945  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>   
1365251727574900738  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>   
1365249991137251328  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>   
1365242894362279938               <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>   
1365242898569134080  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>   
...                                                                                                   ...   
1361903355984023552  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>   
1361917897631805442  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>   
1361912499562553344  <a href="http://twitter.com/download/android" rel="nofollow">Twitter for An

In [42]:
df = df_clean.drop(df_clean[df_clean.text.apply(lambda x: x.isspace())].index)

In [43]:
df_clean[df_clean.text.apply(lambda x: x.isspace())].index

Int64Index([1365363136698679299, 1365461602166857733, 1365202687789305857,
            1365224900827389953, 1365219544663531520, 1365386226317025285,
            1365413761918504961, 1365848396699996163, 1365566690449387520,
            1365681815688798210,
            ...
            1361636627580416000, 1361659683677868040, 1361651186026360835,
            1361610450945802240, 1361837723506589698, 1361734002563043338,
            1361579048204263424, 1361663278204739593, 1361757016700690436,
            1361789837121105922],
           dtype='int64', name='id', length=530)

In [44]:
df_without_duplicate[df_clean.text.apply(lambda x: x.isspace())].to_csv("empty_text.csv")

In [45]:
translated = df_clean.text.swifter.apply(lambda x: GoogleTranslator(source='auto', target='en').translate(str(x)))

TooManyRequests: Server Error: You made too many requests to the server. According to google, you are allowed to make 5 requests per second and up to 200k requests per day. You can wait and try again later or you can try the translate_batch function

In [ ]:
df_clean['translated'] = translated